In [1]:
# !pip install pandas
# !pip install gensim
# !pip install nltk
# !pip install sklearn
# !pip install numpy
# !pip install openpyxl

In [2]:
import pickle
import pandas as pd
import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.cluster import KMeans

import re
import string

import warnings
warnings.simplefilter('ignore')

RANDOM_SEED = 42

# Cleaning The Data - dataset from Deep Blue

With our text data, we are going to apply some of the text pre-processing techniques. Since this cleaning process can go on forever. There's always an exception to every cleaning steps. So, we're going to do this process in a few rounds.

**Below are the steps we will be applying to our dataset:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words
* Stemming & Lemmatization


In [3]:
data_bias=pd.read_excel('assets\pb_spinde.xlsx')
data_bias=data_bias[['article','type']]
data_bias.isnull().values.any()  # check if there are any missing values

True

In [4]:
data_bias.info()  # there are 100 rows has NaN value in column'article'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   article  1600 non-null   object
 1   type     1700 non-null   object
dtypes: object(2)
memory usage: 26.7+ KB


In [5]:
data_bias = data_bias.dropna().reset_index(drop=True)  # decide to drop the rows with missing value
data_bias.head()

,article,type
0,YouTube says no ‘deepfakes’ or ‘birther’ video...,center
1,"FRISCO, Texas — The increasingly bitter disput...",left
2,Speaking to the country for the first time fro...,left
3,A professor who teaches climate change classes...,right
4,The left has a thing for taking babies hostage...,right


In [6]:
# Apply a first round of text cleaning techniques
def clean_text_round1(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

# Let's take a look at the updated text
data_clean = pd.DataFrame(data_bias.article.apply(round1))
data_clean

,article
0,youtube says no ‘deepfakes’ or ‘birther’ video...
1,frisco texas — the increasingly bitter dispute...
2,speaking to the country for the first time fro...
3,a professor who teaches climate change classes...
4,the left has a thing for taking babies hostage...
...,...
1595,the house democrats’ coronavirus recovery bill...
1596,there are many reasons that republicans and co...
1597,a man’s penis becomes a female penis once a ma...
1598,as a selfdescribed democratic socialist sen be...


In [7]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”—…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.article.apply(round2))
data_clean

,article
0,youtube says no deepfakes or birther videos wi...
1,frisco texas the increasingly bitter dispute ...
2,speaking to the country for the first time fro...
3,a professor who teaches climate change classes...
4,the left has a thing for taking babies hostage...
...,...
1595,the house democrats coronavirus recovery bill ...
1596,there are many reasons that republicans and co...
1597,a mans penis becomes a female penis once a man...
1598,as a selfdescribed democratic socialist sen be...


### Removing Stopwords

In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#in case, if we need to remove more stopwords us code below
# stop_words.append('new words')

data_clean['article'] = data_clean['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data_clean

,article
0,youtube says deepfakes birther videos toughene...
1,frisco texas increasingly bitter dispute ameri...
2,speaking country first time oval office tuesda...
3,professor teaches climate change classes subje...
4,left thing taking babies hostage perfect examp...
...,...
1595,house democrats coronavirus recovery bill allo...
1596,many reasons republicans conservative activist...
1597,mans penis becomes female penis man declares t...
1598,selfdescribed democratic socialist sen bernie ...


### Tokenize the sentence

In [9]:
data_clean['article'] = data_clean.article.apply(lambda y: [x for x in word_tokenize(y)])
data_clean

,article
0,"[youtube, says, deepfakes, birther, videos, to..."
1,"[frisco, texas, increasingly, bitter, dispute,..."
2,"[speaking, country, first, time, oval, office,..."
3,"[professor, teaches, climate, change, classes,..."
4,"[left, thing, taking, babies, hostage, perfect..."
...,...
1595,"[house, democrats, coronavirus, recovery, bill..."
1596,"[many, reasons, republicans, conservative, act..."
1597,"[mans, penis, becomes, female, penis, man, dec..."
1598,"[selfdescribed, democratic, socialist, sen, be..."


### Stemming

This process will chops off the ends of words in the hope of achieving this goal correctly most of the time

**(this is optional)**

In [10]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

data_clean_stemmed = data_clean.copy()

data_clean_stemmed['article'] = data_clean_stemmed['article'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
data_clean_stemmed

,article
0,"[youtub, say, deepfak, birther, video, toughen..."
1,"[frisco, texa, increas, bitter, disput, americ..."
2,"[speak, countri, first, time, oval, offic, tue..."
3,"[professor, teach, climat, chang, class, subje..."
4,"[left, thing, take, babi, hostag, perfect, exa..."
...,...
1595,"[hous, democrat, coronavirus, recoveri, bill, ..."
1596,"[mani, reason, republican, conserv, activist, ..."
1597,"[man, peni, becom, femal, peni, man, declar, t..."
1598,"[selfdescrib, democrat, socialist, sen, berni,..."


### Lemmatization

In this process, we use a vocabulary and morphological analysis of words which is aiming to remove inflectional endings only and to return the base or dictionary form of a word

In [11]:
from nltk import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

data_clean_lemma = data_clean.copy()

data_clean_lemma['article']= data_clean_lemma['article'].apply(lemmatize_text)
data_clean_lemma

,article
0,"[youtube, say, deepfakes, birther, video, toug..."
1,"[frisco, texas, increasingly, bitter, dispute,..."
2,"[speaking, country, first, time, oval, office,..."
3,"[professor, teach, climate, change, class, sub..."
4,"[left, thing, taking, baby, hostage, perfect, ..."
...,...
1595,"[house, democrat, coronavirus, recovery, bill,..."
1596,"[many, reason, republican, conservative, activ..."
1597,"[man, penis, becomes, female, penis, man, decl..."
1598,"[selfdescribed, democratic, socialist, sen, be..."


In [12]:
data_cleanned = data_clean_lemma.copy()

data_bias['type'] = data_bias.type.replace({'center':0,'left':-1,'right':1}) # replace the text labels with numbers
data_cleanned['type'] = data_bias['type']  # combine with the cleaned dataset
data_cleanned

,article,type
0,"[youtube, say, deepfakes, birther, video, toug...",0
1,"[frisco, texas, increasingly, bitter, dispute,...",-1
2,"[speaking, country, first, time, oval, office,...",-1
3,"[professor, teach, climate, change, class, sub...",1
4,"[left, thing, taking, baby, hostage, perfect, ...",1
...,...,...
1595,"[house, democrat, coronavirus, recovery, bill,...",1
1596,"[many, reason, republican, conservative, activ...",-1
1597,"[man, penis, becomes, female, penis, man, decl...",1
1598,"[selfdescribed, democratic, socialist, sen, be...",1


NOTE: This data cleaning: **text pre-processing step** could go on for a while, but we are going to stop for now and try it in the modeling part. After that, if we see that the results don't make sense or could be improved, we will come back and make more edits

**More data cleaning steps after tokenization:**
* Stemming 
* Parts of speech tagging
* Create bi-grams or tri-grams (such as 'thank you' into one term)
* Deal with typos
* And more...

In [13]:
# Let's pickle it for later use
data_cleanned.to_pickle("data_cleanned_corpus.pkl")

# Continuing on the Model part (copied from Jame's code)
## Part 1a: Political Bias Modeling

First we want to build a model of political bias using features that will be available in our primary dataset. We'll import the Spinde political bias dataset and select the article text and bias rating columns. Then, we'll vectorize the article text and train the model.

In [14]:
pb_reduced = data_bias.copy()
pb_reduced['tokens'] = data_cleanned['article']
pb_reduced

,article,type,tokens
0,YouTube says no ‘deepfakes’ or ‘birther’ video...,0,"[youtube, say, deepfakes, birther, video, toug..."
1,"FRISCO, Texas — The increasingly bitter disput...",-1,"[frisco, texas, increasingly, bitter, dispute,..."
2,Speaking to the country for the first time fro...,-1,"[speaking, country, first, time, oval, office,..."
3,A professor who teaches climate change classes...,1,"[professor, teach, climate, change, class, sub..."
4,The left has a thing for taking babies hostage...,1,"[left, thing, taking, baby, hostage, perfect, ..."
...,...,...,...
1595,The House Democrats’ coronavirus recovery bill...,1,"[house, democrat, coronavirus, recovery, bill,..."
1596,There are many reasons that Republicans and co...,-1,"[many, reason, republican, conservative, activ..."
1597,A man’s penis becomes a female penis once a ma...,1,"[man, penis, becomes, female, penis, man, decl..."
1598,"As a self-described Democratic socialist, Sen....",1,"[selfdescribed, democratic, socialist, sen, be..."


In [15]:
#Now we'll train the Word2Vec model on our text tokens.
wv_mod = Word2Vec(pb_reduced['tokens'], seed = RANDOM_SEED)
wv_mod.save("models/pb_w2v_model.pkl")

In [16]:
#We'll extract the vectors from the model...
vectors = wv_mod.wv
#...and since each word is a vector of 100 numbers, we'll take the mean of all word vectors in a given article 
#to represent the article as a whole
vec_frame = pd.DataFrame([vectors.get_mean_vector(x) for x in pb_reduced.tokens])

In [17]:
#Finally, we'll train a Random Forest classifier on the vectorized text to predict article bias.
X_train, X_test, y_train, y_test = train_test_split(vec_frame, pb_reduced.type, test_size=0.2, random_state=RANDOM_SEED)

In [18]:
clf = RandomForestClassifier(random_state=RANDOM_SEED)
clf.fit(X_train, y_train)
filename = "models/pb_classifier_model.pkl"
pickle.dump(clf, open(filename, 'wb'))
clf.score(X_test, y_test)

0.8125

# Cleaning The Data - dataset from Kaggle

In [19]:
data_news=pd.read_csv('assets/fn_kagg_train.csv')
data_news.isnull().values.any()  # check if there are any missing values

True

In [20]:
data_news.isna().any()[lambda x: x] # check which column has missing values

title     True
author    True
text      True
dtype: bool

In [21]:
print ('Total number of rows',  len(data_news))
print('Number of title with missing value', data_news['title'].isna().sum())
print('Number of author with missing value', data_news['author'].isna().sum())
print('Number of text with missing value', data_news['text'].isna().sum())

# it appears that the rows with miss value only has a small number, decide to remove
data_news = data_news.dropna().reset_index(drop=True)

Total number of rows 20800
Number of title with missing value 558
Number of author with missing value 1957
Number of text with missing value 39


In [22]:
data_news

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
18280,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
18281,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
18282,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
18283,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


### Applying the same cleaning rounds from previous code
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words
* Stemming & Lemmatization


In [23]:
data_news_clean = pd.DataFrame(data_news.text.apply(round1))
data_news_clean = pd.DataFrame(data_news_clean.text.apply(round2))

More cleaning are required 

**After examine the 'text' columns, it appears that some characters are encoded with UTF-8 format, and ascii value
(such as Ã¼, Ð», â€“.etc)**

In [24]:
# Apply a third round of cleaning

'''Remove some UTF-8 format characters and converting the ascii value which missed from previous rounds'''
data_news_clean['text'] = data_news_clean['text'].apply(lambda x: x.encode('ascii', 'ignore').decode("ascii","ignore"))
data_news_clean

,text
0,house dem aide we didnt even see comeys letter...
1,ever get the feeling your life circles the rou...
2,why the truth might get you fired october th...
3,videos civilians killed in single us airstrik...
4,print an iranian woman has been sentenced to s...
...,...
18280,rapper t i unloaded on black celebrities who m...
18281,when the green bay packers lost to the washing...
18282,the macys of today grew from the union of seve...
18283,nato russia to hold parallel exercises in balk...


### Removing Stop, Tokenization, Lemmatization

In [25]:
# this process will run a bit longer since we are doing all three together 

data_news_clean['text'] = data_news_clean['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data_news_clean['text'] = data_news_clean.text.apply(lambda y: [x for x in word_tokenize(y)])
data_news_lemma = data_news_clean.copy()
data_news_lemma['text']= data_news_lemma['text'].apply(lemmatize_text)

In [26]:
# Let's pickle it for later use
data_news_lemma.to_pickle("data_news_lemma_corpus.pkl")

# Continuing on the Model part (copied from Jame's code)
## Part 1b: Applying the Model

Now, we want to predict the political bias of the target fake news dataset. We'll save these predictions as probabilities, which we'll use as additional features for clustering and trustworthiness prediction.

In [27]:
fn_kag_tok = data_news.copy()
fn_kag_tok['text_tokens'] = data_news_lemma['text']

In [28]:
#Some articles have very few words, so we'll drop any rows with fewer than 30 tokens.
fn_kag_tok['tmp'] = fn_kag_tok['text_tokens'].apply(lambda x: len(x))
fn_kag_tok = fn_kag_tok[fn_kag_tok['tmp']>30]
fn_kag_tok = fn_kag_tok.drop(columns='tmp')
fn_kag_tok

,id,title,author,text,label,text_tokens
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"[house, dem, aide, didnt, even, see, comeys, l..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"[ever, get, feeling, life, circle, roundabout,..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,"[truth, might, get, fired, october, tension, i..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,"[video, civilian, killed, single, u, airstrike..."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,"[print, iranian, woman, sentenced, six, year, ..."
...,...,...,...,...,...,...
18280,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,"[rapper, unloaded, black, celebrity, met, dona..."
18281,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"[green, bay, packer, lost, washington, redskin..."
18282,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,"[macys, today, grew, union, several, great, na..."
18283,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"[nato, russia, hold, parallel, exercise, balka..."


In [29]:
#Now we'll apply the Word2Vec model we generated above to our tokens to vectorize the text.
vec_frame = pd.DataFrame([vectors.get_mean_vector(x) for x in fn_kag_tok.text_tokens])

In [30]:
#Now we apply the Random Forest classifier to our vectorized text and save out the predicted probabilities.
preds = pd.DataFrame(clf.predict_proba(vec_frame), columns=['dem_bias','neutral','rep_bias'])

In [31]:
#Finally, we'll rejoin the predictions to the original dataset.
fn_kag_reduced = fn_kag_tok.copy().reset_index(drop=True)
fn_kag_reduced = fn_kag_reduced.join(preds)
fn_kag_reduced

,id,title,author,text,label,text_tokens,dem_bias,neutral,rep_bias
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"[house, dem, aide, didnt, even, see, comeys, l...",0.49,0.02,0.49
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"[ever, get, feeling, life, circle, roundabout,...",0.52,0.01,0.47
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,"[truth, might, get, fired, october, tension, i...",0.53,0.08,0.39
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,"[video, civilian, killed, single, u, airstrike...",0.24,0.19,0.57
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,"[print, iranian, woman, sentenced, six, year, ...",0.47,0.17,0.36
...,...,...,...,...,...,...,...,...,...
17487,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,"[rapper, unloaded, black, celebrity, met, dona...",0.47,0.05,0.48
17488,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"[green, bay, packer, lost, washington, redskin...",0.23,0.22,0.55
17489,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,"[macys, today, grew, union, several, great, na...",0.31,0.13,0.56
17490,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"[nato, russia, hold, parallel, exercise, balka...",0.28,0.25,0.47


# Continuing on the Model part (copied from Jame's code)
## Part 2: Clustering
Once we have all the features we want, we'll do unsupervised clustering. Ideally we'd want to do some evaluations to find an ideal number of clusters, but for now we'll just go with 4.

We'll need to re-vectorize the text, as the political bias vectors won't work here. Also, we'd probably want to vectorize both headline and article body, but for now I'll just vectorize the article body.

In [32]:
#From this point on, we should be concerned with data leakage. Everything prior to now could in theory be applied to live data. We'll go ahead and split the data out into train and test sets.

X_train, X_test, y_train, y_test = train_test_split(fn_kag_reduced.drop(columns=['label']), fn_kag_reduced.label, test_size=0.2, random_state=RANDOM_SEED)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [33]:
#Since we already have the tokenized text from above, we can just go ahead and train the new Word2Vec model on those tokens.
wv_mod = Word2Vec(X_train['text_tokens'], seed = RANDOM_SEED)
wv_mod.save("models/fn_w2v_model.pkl")

In [34]:
#Again we'll extract and average the word vectors.
vectors = wv_mod.wv
vec_frame = pd.DataFrame([vectors.get_mean_vector(x) for x in X_train.text_tokens])

In [35]:
#We'll join the new word vectors with the bias estimates we generate above.
X_train_all = vec_frame.join(X_train).drop(columns=['id','title','author','text','text_tokens'])

In [36]:
#Finally we'll build our clustering model...
cls = KMeans(4, random_state=RANDOM_SEED).fit(X_train_all)
filename = "models/cluster_mod.pkl"
pickle.dump(cls, open(filename, 'wb'))

In [37]:
#...and add the predicted clusters back into the vector dataframe.
X_train_all['cluster'] = cls.predict(X_train_all)
X_train_all

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,dem_bias,neutral,rep_bias,cluster
0,-0.009658,0.005730,0.042834,0.046615,-0.047032,0.023168,0.037267,0.036255,-0.016245,-0.064646,...,0.019453,0.015911,-0.018045,0.006621,0.003714,-0.032222,0.340000,0.12,0.540000,0
1,0.035299,-0.013338,0.016556,-0.008684,-0.026056,-0.013706,0.007896,0.012592,-0.011828,-0.098827,...,0.022860,0.019518,0.025123,0.037479,-0.004328,0.042627,0.410000,0.14,0.450000,2
2,-0.008059,0.002088,0.008326,0.015817,-0.005587,-0.045881,0.029630,-0.020504,-0.021675,-0.053884,...,0.011687,0.039100,-0.013914,0.016798,0.011421,0.013591,0.330000,0.12,0.550000,0
3,-0.013599,-0.020696,0.048159,0.041106,-0.050609,0.007193,0.032249,-0.023648,-0.026226,-0.074807,...,0.010859,0.024434,0.011079,0.023782,0.025636,0.011899,0.470000,0.11,0.420000,2
4,0.010908,-0.023946,0.039867,0.039451,-0.071986,0.025116,0.054771,-0.024438,0.041491,-0.095228,...,0.033196,-0.005189,0.013563,0.037562,0.010634,-0.005792,0.348000,0.24,0.412000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13988,-0.064868,-0.106135,-0.094492,-0.074123,-0.019983,0.037429,0.000613,-0.220459,0.085949,0.172327,...,0.024090,-0.058535,-0.052388,-0.091403,-0.074858,-0.058445,0.237500,0.36,0.402500,3
13989,0.009163,-0.029318,0.051790,0.034272,-0.045128,0.036544,-0.007169,0.028784,-0.014257,-0.076432,...,0.024975,0.016809,0.008687,0.018483,-0.002483,-0.008261,0.420000,0.16,0.420000,2
13990,0.010447,-0.019237,0.020061,0.006971,-0.033006,-0.022863,0.033802,0.031342,-0.021314,-0.098133,...,0.001802,0.007989,-0.009117,0.009067,0.001602,-0.004169,0.420000,0.17,0.410000,1
13991,-0.028158,-0.016735,0.028893,0.067358,-0.007451,0.041093,0.004991,0.049816,-0.057361,-0.043732,...,0.037840,0.015241,0.006647,-0.001574,-0.044859,-0.015713,0.327333,0.25,0.422667,1


## Part 3: Supervised Learning

Now that we have all of our features and clusters, and article body text is already vectorized, we can train a classifier to predict whether a given article is misinformation or not.

In [38]:
#We need to apply the vectorization and clustering from above to the test data.
vec_frame = pd.DataFrame([vectors.get_mean_vector(x) for x in X_test.text_tokens])
X_test_all = vec_frame.join(X_test).drop(columns=['id','title','author','text','text_tokens'])
X_test_all['cluster'] = cls.predict(X_test_all)
X_test_all

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,dem_bias,neutral,rep_bias,cluster
0,-0.004069,0.006721,0.025754,0.016396,-0.024229,0.014822,-0.005376,0.004720,-0.014789,-0.061167,...,0.000795,0.015933,0.034293,0.010748,-0.005926,0.024244,0.54,0.08,0.38,2
1,0.007615,0.021091,0.009161,0.035196,-0.026442,0.019393,0.025404,0.076108,-0.016611,-0.025038,...,0.039519,0.040006,0.030866,0.032353,-0.008771,-0.000636,0.59,0.02,0.39,2
2,-0.015921,-0.017781,0.027126,0.039706,-0.072932,-0.000181,0.049133,-0.028249,0.002583,-0.090377,...,0.006657,0.026980,-0.000872,0.036489,0.011354,-0.001185,0.29,0.30,0.41,1
3,0.026510,0.011171,-0.015315,0.002247,0.002525,0.007189,-0.045030,0.004801,-0.010983,-0.036082,...,0.050313,0.056651,-0.007929,0.030120,-0.000486,0.025396,0.32,0.18,0.50,1
4,0.030441,0.015128,0.020871,0.011121,-0.068780,0.009686,0.026041,0.025270,0.028207,-0.053540,...,0.010415,0.031614,0.007730,0.036058,0.023472,0.025822,0.48,0.07,0.45,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,0.000622,-0.015777,0.001780,0.022281,-0.019934,0.022051,0.005040,0.027889,-0.018690,-0.037581,...,0.043006,0.008060,0.006654,0.009844,-0.018182,0.003670,0.54,0.06,0.40,2
3495,-0.003556,-0.001852,0.027409,0.018975,-0.044493,-0.012646,-0.015225,-0.051780,-0.019676,-0.039484,...,0.017797,0.033542,0.000850,-0.028908,-0.022689,-0.023097,0.12,0.28,0.60,1
3496,-0.005016,-0.026904,-0.028986,-0.009276,-0.035803,-0.053517,0.012711,-0.000715,-0.050154,-0.062212,...,0.025777,-0.008230,-0.005290,0.012638,-0.013910,-0.047990,0.13,0.24,0.63,1
3497,-0.003582,-0.017635,0.022182,0.072596,-0.027446,0.024533,-0.008718,0.059137,-0.027213,-0.041577,...,0.051082,0.046678,0.002067,0.037541,-0.034289,-0.009565,0.56,0.06,0.38,2


In [39]:
clf = RandomForestClassifier(random_state=RANDOM_SEED)
clf.fit(X_train_all, y_train)
filename = "models/fn_classifier_model.pkl"
pickle.dump(clf, open(filename, 'wb'))
clf.score(X_test_all, y_test)

0.8939697056301801

## Part 4: Export CSV

So now we have 2 datasets (3 if you count the unused 'test' set from the Kaggle fake news data). We want to have 1 large dataset to power the dashboard. Let's load each dataset into a dataframe, vectorize the text, predict a cluster, and predict whether it's misinformation or not. We'll start fresh to keep things simple, and we'll just use article body text.

This part can really stand on its own... should probably pull it out into a separate notebook/script? Only issue is we'd have to repeat the code for the NLP steps, unless we save them into their own script as well.

In [40]:
#Load each datafile we want to process
data_bias=pd.read_excel('assets/pb_spinde.xlsx')
data_news_1=pd.read_csv('assets/fn_kagg_train.csv')
data_news_2=pd.read_csv('assets/fn_kagg_test.csv')

In [41]:
#Load all of our models
pb_vec_mod = Word2Vec.load('models/pb_w2v_model.pkl')
fn_vec_mod = Word2Vec.load('models/fn_w2v_model.pkl')
cls = pickle.load(open('models/cluster_mod.pkl', 'rb'))
pb_clf = pickle.load(open('models/pb_classifier_model.pkl', 'rb'))
fn_clf = pickle.load(open('models/fn_classifier_model.pkl', 'rb'))

In [42]:
#Grab just the article text, drop empty cells, and stack the dataframes.
db_text = data_bias.article
dn1_text = data_news_1.text
dn2_text = data_news_2.text
full_data = pd.DataFrame(pd.concat([db_text, dn1_text, dn2_text], axis = 0).dropna().reset_index(drop=True))
full_data.columns = ['full_text']
full_data

,full_text
0,YouTube says no ‘deepfakes’ or ‘birther’ video...
1,"FRISCO, Texas — The increasingly bitter disput..."
2,Speaking to the country for the first time fro...
3,A professor who teaches climate change classes...
4,The left has a thing for taking babies hostage...
...,...
27549,Of all the dysfunctions that plague the world’...
27550,WASHINGTON — Gov. John Kasich of Ohio on Tu...
27551,Good morning. (Want to get California Today by...
27552,« Previous - Next » 300 US Marines To Be Deplo...


In [43]:
#Now we'll apply all the NLP steps from above to the full series. This can take a while. I'm skipping the lemmatization and stemming for now...
# full_data_clean = full_data.copy()
# full_data_clean['processing'] = full_data_clean.full_text.apply(lambda x: x.encode('ascii', 'ignore').decode("ascii","ignore"))
# full_data_clean['processing'] = full_data_clean.processing.apply(round1)
# full_data_clean['processing'] = full_data_clean.processing.apply(round2)
# full_data_clean['processing'] = full_data_clean.processing.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# full_data_clean['tokens'] = full_data_clean.processing.apply(lambda y: [x for x in word_tokenize(y)])

# full_data_clean['tmp'] = full_data_clean['tokens'].apply(lambda x: len(x))
# full_data_clean = full_data_clean[full_data_clean['tmp']>30]
# full_data_clean = full_data_clean.drop(columns=['tmp','processing'])

# full_data_clean.to_pickle('clean_full_data.pkl')
full_data_clean = pd.read_pickle('clean_full_data.pkl').reset_index(drop=True)
full_data_clean

,full_text,tokens
0,YouTube says no ‘deepfakes’ or ‘birther’ video...,"[youtube, says, deepfakes, birther, videos, to..."
1,"FRISCO, Texas — The increasingly bitter disput...","[frisco, texas, increasingly, bitter, dispute,..."
2,Speaking to the country for the first time fro...,"[speaking, country, first, time, oval, office,..."
3,A professor who teaches climate change classes...,"[professor, teaches, climate, change, classes,..."
4,The left has a thing for taking babies hostage...,"[left, thing, taking, babies, hostage, perfect..."
...,...,...
25834,Of all the dysfunctions that plague the world’...,"[dysfunctions, plague, worlds, megacities, non..."
25835,WASHINGTON — Gov. John Kasich of Ohio on Tu...,"[washington, gov, john, kasich, ohio, tuesday,..."
25836,Good morning. (Want to get California Today by...,"[good, morning, want, get, california, today, ..."
25837,« Previous - Next » 300 US Marines To Be Deplo...,"[previous, next, us, marines, deployed, russia..."


In [44]:
#Now we'll use the pb vectorizer and generate political bias predictions for the whole dataset.
vectors = pb_vec_mod.wv
vec_frame = pd.DataFrame([vectors.get_mean_vector(x) for x in full_data_clean.tokens])
preds = pd.DataFrame(pb_clf.predict_proba(vec_frame), columns=['dem_bias','neutral','rep_bias'])
full_data_clean = full_data_clean.join(preds)
full_data_clean

,full_text,tokens,dem_bias,neutral,rep_bias
0,YouTube says no ‘deepfakes’ or ‘birther’ video...,"[youtube, says, deepfakes, birther, videos, to...",0.350000,0.33,0.320000
1,"FRISCO, Texas — The increasingly bitter disput...","[frisco, texas, increasingly, bitter, dispute,...",0.490000,0.12,0.390000
2,Speaking to the country for the first time fro...,"[speaking, country, first, time, oval, office,...",0.730000,0.08,0.190000
3,A professor who teaches climate change classes...,"[professor, teaches, climate, change, classes,...",0.520000,0.02,0.460000
4,The left has a thing for taking babies hostage...,"[left, thing, taking, babies, hostage, perfect...",0.140000,0.01,0.850000
...,...,...,...,...,...
25834,Of all the dysfunctions that plague the world’...,"[dysfunctions, plague, worlds, megacities, non...",0.310000,0.06,0.630000
25835,WASHINGTON — Gov. John Kasich of Ohio on Tu...,"[washington, gov, john, kasich, ohio, tuesday,...",0.320000,0.38,0.300000
25836,Good morning. (Want to get California Today by...,"[good, morning, want, get, california, today, ...",0.190000,0.18,0.630000
25837,« Previous - Next » 300 US Marines To Be Deplo...,"[previous, next, us, marines, deployed, russia...",0.326923,0.05,0.623077


In [45]:
#Now we'll predict clusters using the fake news w2v model.
vectors = fn_vec_mod.wv
vec_frame = pd.DataFrame([vectors.get_mean_vector(x) for x in full_data_clean.tokens])
cluster_frame = vec_frame.join(full_data_clean).drop(columns=['full_text','tokens'])
full_data_clean['cluster'] = cls.predict(cluster_frame)
full_data_clean

,full_text,tokens,dem_bias,neutral,rep_bias,cluster
0,YouTube says no ‘deepfakes’ or ‘birther’ video...,"[youtube, says, deepfakes, birther, videos, to...",0.350000,0.33,0.320000,1
1,"FRISCO, Texas — The increasingly bitter disput...","[frisco, texas, increasingly, bitter, dispute,...",0.490000,0.12,0.390000,2
2,Speaking to the country for the first time fro...,"[speaking, country, first, time, oval, office,...",0.730000,0.08,0.190000,2
3,A professor who teaches climate change classes...,"[professor, teaches, climate, change, classes,...",0.520000,0.02,0.460000,2
4,The left has a thing for taking babies hostage...,"[left, thing, taking, babies, hostage, perfect...",0.140000,0.01,0.850000,0
...,...,...,...,...,...,...
25834,Of all the dysfunctions that plague the world’...,"[dysfunctions, plague, worlds, megacities, non...",0.310000,0.06,0.630000,0
25835,WASHINGTON — Gov. John Kasich of Ohio on Tu...,"[washington, gov, john, kasich, ohio, tuesday,...",0.320000,0.38,0.300000,1
25836,Good morning. (Want to get California Today by...,"[good, morning, want, get, california, today, ...",0.190000,0.18,0.630000,0
25837,« Previous - Next » 300 US Marines To Be Deplo...,"[previous, next, us, marines, deployed, russia...",0.326923,0.05,0.623077,0


In [49]:
#Finally, we'll generate predicted fake news probabilities using our fn classifier.
class_frame = vec_frame.join(full_data_clean).drop(columns=['full_text','tokens'])
preds = pd.DataFrame(fn_clf.predict_proba(class_frame), columns=['not_misinfo', 'misinfo'])
full_data_clean = full_data_clean.join(preds)
full_data_clean


,full_text,tokens,dem_bias,neutral,rep_bias,cluster,not_misinfo,misinfo
0,YouTube says no ‘deepfakes’ or ‘birther’ video...,"[youtube, says, deepfakes, birther, videos, to...",0.350000,0.33,0.320000,1,0.55,0.45
1,"FRISCO, Texas — The increasingly bitter disput...","[frisco, texas, increasingly, bitter, dispute,...",0.490000,0.12,0.390000,2,0.98,0.02
2,Speaking to the country for the first time fro...,"[speaking, country, first, time, oval, office,...",0.730000,0.08,0.190000,2,0.83,0.17
3,A professor who teaches climate change classes...,"[professor, teaches, climate, change, classes,...",0.520000,0.02,0.460000,2,0.48,0.52
4,The left has a thing for taking babies hostage...,"[left, thing, taking, babies, hostage, perfect...",0.140000,0.01,0.850000,0,0.27,0.73
...,...,...,...,...,...,...,...,...
25834,Of all the dysfunctions that plague the world’...,"[dysfunctions, plague, worlds, megacities, non...",0.310000,0.06,0.630000,0,0.75,0.25
25835,WASHINGTON — Gov. John Kasich of Ohio on Tu...,"[washington, gov, john, kasich, ohio, tuesday,...",0.320000,0.38,0.300000,1,0.93,0.07
25836,Good morning. (Want to get California Today by...,"[good, morning, want, get, california, today, ...",0.190000,0.18,0.630000,0,0.93,0.07
25837,« Previous - Next » 300 US Marines To Be Deplo...,"[previous, next, us, marines, deployed, russia...",0.326923,0.05,0.623077,0,0.16,0.84


In [51]:
full_data_clean.to_csv("assets/all_predictions.csv")

## To-Do:
1. Put all NLP stuff in a function, so that we can just pass a column of text and get the clean tokens out.
2. (Maybe?) Clean the notebook up into two scripts: one that builds and pickles the models, and one that takes a csv (or set of csvs/excels/whatever), runs all the models, and outputs a csv with all the data needed for the dashboard.
3. ??Include headline text in models
4. Do some deeper evaluations on the clustering and supervised learning portions (supervised portion is not critical, we have good accuracy right now).